## 주제

주식 종료 가격 예측

## 데이터

### 데이터 설명

- Stock_List.csv: 종목 번호 데이터
    - 종목명: 주식 종목 명
    - 종목코드: 주식 종목 코드번호
    - 상장시장: 주식 종목 상장 시장 (KOSPI or KOSDAQ)

### 전처리

- 시계열 분해를 통해 종가 추세와 계절성 데이터를 추가
- 종가에 대하여 5일, 20일, 60일, 120일, 240일 이동평균을 추가
- 최종적으로 일주일씩 (business day 기준으로 5일씩) lag features를 생성

## 코드 흐름

### 전처리

- 기본 날짜 프레임 생성
    - 2011년 1월 1일부터 2021년 9월 17일까지의 영업일(Business day) 기준 날짜 생성
    - 연도, 월, 주, 일, 요일, 월별 주차 등의 시간 관련 파생 변수 추가
- 주가 데이터 수집 및 병합
    - 각 종목 코드에 대해 주가 데이터 불러오기
    - 날짜 기준으로 병합하고, 공휴일 등 결측치는 전날 값으로 보간
    - 상장 전 기간의 결측치는 drop
- 시계열 분해 (Trend / Seasonal 분리)
    - 종가 데이터를 시계열 분해
    - 추세와 계절성 값을 새로운 칼럼으로 추가
- 이동평균(rolling mean) 추가
    - 종가에 대해 5일, 20일, 60일, 120일, 240일 이동평균선 추가
    - 단기/중기/장기 추세를 반영한 특징 생성
- Lag(시차) 피처 생성
    - 주요 수치형 칼럼들을 5일 전 값으로 이동(lag)
    - 과거 정보(5영업일 전)를 현재 예측에 활용할 수 있도록 변환
- 정리 및 저장
    - 인덱스 초기화 후, 최종 전처리된 종목별 DataFrame을 stock_df_dict[stock_code]에 저장

### 모델링 (XGBoost)

- SEED 고정
    - 모델의 재현성을 위해 랜덤 시드를 고정한다.
- NMAE 함수 정의
    - Normalized Mean Absolute Error(정규화된 평균 절대 오차) 계산 함수
    - 예측 오차를 실제값으로 나눠서 상대적인 오차 비율(%)로 평가
- 반복 학습 구조
    - stock_df_dict에 있는 모든 종목에 대해 반복 학습 수행
    - 각 종목별 모델을 따로 학습하고, 예측 결과를 저장
- 데이터 분할 (Public / Private 구간)
    - 각 종목별 데이터를 시계열 순서로 분할
    - Public
        - train: 전체 중 마지막 15일 제외
        - test: 마지막 15일 중 최근 10~15일 구간
        - valid: train의 마지막 5일
    - Private
        - train: 전체 중 마지막 5일 제외 구간
        - test: 마지막 5일
        - valid: train의 마지막 5일
- 모델 학습
    - XGBoost 회귀 모델 사용
    - 파라미터 설정
        - n_estimators=10000: 최대 트리 개수
        - learning_rate=0.05: 학습률 (낮게→천천히 학습)
        - tree_method=’gpu_hist’: GPU 가속
        - early_stopping_rounds=100: 검증 손실이 100회 연속 개선되지 않으면 조기 종료
        - eval_metric=’mae’: 평가 지표로 MAE 사용
- 예측 및 성능 측정
    - xgb_oof: 검증 구간(최근 5일)의 예측값
    - xgb_pred: 테스트 구간의 예측값
    - 각 종목별 예측 결과를 xgb_pred_dict에 저장
- NMAE 계산 및 출력
    - 검증 구간의 NMAE 점수를 계산해 모델 성능을 평가
    - 각 종목의 점수를 리스트에 저장
- 최종 평균 성능 출력
    - 모든 종목의 검증 결과를 평균 내어, 모델의 전체적인 성능을 요약

## 차별점 및 배울점

- 종목별 독립 모델링
    - 모든 종목을 하나로 묶어 모델링하지 않고, 종목별로 개별 데이터프레임+개별 모델을 학습
    - 종목별 특성이 반영되어 맞춤형 예측 가능
- 시계열 기반 전처리
    - rolling mean과 lag features를 활용
    - 종가 trend와 seasonal 분해 후 피처화
    - 단순 회귀/트리 모델에 시계열 패턴을 반영
- 결측치 처리
    - 공휴일, 휴장일은 전일 값으로 보간
    - 상장 전 데이터는 제거
- 평가
    - 정규화된 오차율(NMAE)로 평가→종목별 가격 차이가 큰 경우에도 공정한 평가 가능